In [1]:
import PyPDF3
import pandas as pd
import numpy as np
import string
import re

In [2]:
# Membaca teks di dalam dokumen PDF

jurnalFile = open('jurnal/1.pdf', 'rb')

pdfReader = PyPDF3.PdfFileReader(jurnalFile)    

text = ""

for page in pdfReader.pages:
    text += page.extractText()
    
jurnalFile.close()

print(text)

E
-
ISSN : 2775
-
412X
!
JURNAL 
SINTAKS LOGIKA
 
Vol. 
1 
 
No. 
3
 
, 
September
 
-
 
20
21
 
 
https://jurnal.umpar.ac.id/index.php/sylog
 
 
 
DOI: 
https://doi.
org/10.31850/jsilog.v1i
3
 
 
 
!
 
1
!
SISTEM INFORMASI PENGADUAN PELANGGAN PADA 
 
PT. PLN (PERSERO) ULP PANGSID UP 3 PAREPARE BERBASIS WEB
 
 
!
Ade Hastuty
1
, 
Ahmad Selao
2
, 
Fergiawan HR
.
 
Saleh
3
 
1,
2,3
Program Studi 
...
, Universitas Muhammadiyah Parepare, Indonesia
 
fergiawanhr.saleh95@gmail.com
 
!
Informasi Artikel
 
 
ABSTRACT 
 
Riwayat Artikel:
 
Dikirim
 
Author
 
: 
9
-
01
-
2021
 
Diterima
 
Redaksi 
: 
10
-
01
-
2021
 
Revisi
 
Reviewer
: 
12
-
01
-
2021
 
Diterbitkan 
online
: 
18
-
01
-
2021
 
 
 
This study takes the problem of the PLN Customer Complaint Application. This research 
provides an overview of the accuracy in the process of submitting custom
er complaints 
and complaints by using a database
-
based system. Customer complaints are currently 
still manual, namely coming directly to t

In [3]:
## fungsi untuk menghapus url didalam teks
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",text)

## fungsi untuk menghapus tanda baca didalam teks
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

## fungsi untuk mengubah teks menjadi huruf kecil
def case_folding(text):
    return text.casefold()

In [4]:
## stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

stopwords = StopWordRemoverFactory().create_stop_word_remover()

def remove_stopwords(text):
    return stopwords.remove(text)

In [5]:
## stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stemmer = StemmerFactory().create_stemmer()

def stemming(text):
    return stemmer.stem(text)

In [6]:
# Preprocessing

text = remove_URL(text)
text = remove_punct(text)
text = case_folding(text)
text = remove_stopwords(text)
text = stemming(text)

print(text)

e issn 2775 412x jurnal sintaks logika vol 1 no 3 september 20 21 doi org1031850jsilogv1i 3 1 sistem informasi adu langgan pt pln persero ulp pangsid up 3 parepare bas web ade hastuty 1 ahmad selao 2 fergiawan hr saleh 3 1 23 program studi universitas muhammadiyah parepare indonesia fergiawanhrsaleh95gmailcom informasi artikel abstract riwayat artikel kirim author 9 01 2021 terima redaksi 10 01 2021 revisi reviewer 12 01 2021 terbit online 18 01 2021 this study takes the problem of the pln customer complaint application this research provides an overview of the accuracy in the process of submitting custom er complaints and complaints by using a database based system customer complaints are currently still manual namely coming directly to the pln office or using tel 123 a condition that occurs is the slow handling due to manual recording as a result the response back to a complaint or complaint from customers tends not to be recorded in detail because it has not utilized data integrated

In [7]:
# Memecah setiap kata

keywords = re.findall(r'[a-zA-Z]\w+',text)

# Jumlah kata
len(keywords)

2108

In [20]:
# Filter data yang sama dan menyimpan di DataFrame

df = pd.DataFrame(list(set(keywords)),columns=['keywords'])

df['keywords']

0      universitas
1             ajar
2               st
3             kota
4            lapor
          ...     
730         bahasa
731            isi
732         office
733             to
734            ata
Name: keywords, Length: 735, dtype: object

In [9]:
def weightage(word,text,number_of_documents=1):
    word_list = re.findall(word,text)
    number_of_times_word_appeared =len(word_list)
    tf = number_of_times_word_appeared/float(len(text))
    idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
    tf_idf = tf*idf
    return number_of_times_word_appeared,tf,idf ,tf_idf  

In [17]:
df['total kemunculan'] = df['keywords'].apply(lambda x: weightage(x,text)[0])
df['tf'] = df['keywords'].apply(lambda x: weightage(x,text)[1])
df['idf'] = df['keywords'].apply(lambda x: weightage(x,text)[2])
df['tf_idf'] = df['keywords'].apply(lambda x: weightage(x,text)[3])

In [11]:
df = df.sort_values('tf_idf',ascending=True)
df.to_csv('Keywords.csv')
df.head(100)

,keywords,total kemunculan,tf,idf,tf_idf
632,an,336,0.022840,-5.817111,-0.132863
453,as,204,0.013867,-5.318120,-0.073747
606,at,193,0.013119,-5.262690,-0.069044
511,in,181,0.012304,-5.198497,-0.063961
237,er,138,0.009381,-4.927254,-0.046221
...,...,...,...,...,...
95,hasil,10,0.000680,-2.302585,-0.001565
338,buat,10,0.000680,-2.302585,-0.001565
320,test,10,0.000680,-2.302585,-0.001565
642,laku,10,0.000680,-2.302585,-0.001565


In [22]:
df['keywords'][0]

'universitas'